In [1]:
import warnings
warnings.filterwarnings("ignore") 

In [16]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from datetime import timedelta,datetime
df=pd.read_csv('C:\\workplace\\electricity_demand\\dataset\\AEP_hourly.csv')
df.sort_values('Datetime',ascending=True,inplace=True)
df.columns=['date_time','aep_mw']

In [36]:
def preparion (df,num_lags,lag_feature):
    df_prpocessed=pd.DataFrame(index=df.index,columns=['date_time','cos_day_year','sin_day_year',
                                                       'cos_day_week','sin_day_week','cos_hour','sin_hour']+['lags_{}'.format(i) for i in range(num_lags)]+['target'])
    df_prpocessed['target']=df[lag_feature].shift(-1)
    for i in range(num_lags):
        df_prpocessed['lags_{}'.format(i)]=df[lag_feature].shift(i)

    df_prpocessed['date_time']=df['date_time']

    a=pd.Series(pd.to_datetime(df_prpocessed['date_time']))
    df_prpocessed['cos_day_year']=pd.Series(np.cos(2*np.pi*a.dt.day_of_year/366),index=df_prpocessed.index)
    df_prpocessed['sin_day_year']=pd.Series(np.sin(2*np.pi*a.dt.day_of_year/366),index=df_prpocessed.index)
    df_prpocessed['cos_day_week']=pd.Series(np.cos(2*np.pi*a.dt.day/7),index=df_prpocessed.index)
    df_prpocessed['sin_day_week']=pd.Series(np.sin(2*np.pi*a.dt.day/7),index=df_prpocessed.index)
    df_prpocessed['cos_hour']=pd.Series(np.cos(2*np.pi*a.dt.hour/24),index=df_prpocessed.index)
    df_prpocessed['sin_hour']=pd.Series(np.sin(2*np.pi*a.dt.hour/24),index=df_prpocessed.index)
    
    df_prpocessed.dropna(inplace=True)
    df_prpocessed.index=df_prpocessed['date_time']
    df_prpocessed.drop('date_time',axis=1,inplace=True)
    return df_prpocessed

In [58]:
train_df=df[df['date_time']<='2017-08-02 23:00:00']
train_df=preparion(train_df,500,'aep_mw')
train_df

,cos_day_year,sin_day_year,cos_day_week,sin_day_week,cos_hour,sin_hour,lags_0,lags_1,lags_2,lags_3,...,lags_491,lags_492,lags_493,lags_494,lags_495,lags_496,lags_497,lags_498,lags_499,target
date_time,,,,,,,,,,,,,,,,,,,,,
2004-10-21 20:00:00,0.344707,-0.938710,1.000000,-7.347881e-16,5.000000e-01,-0.866025,15921.0,15340.0,14988.0,14943.0,...,14903.0,14618.0,13692.0,12280.0,11681.0,11597.0,11692.0,11935.0,12379.0,15886.0
2004-10-21 21:00:00,0.344707,-0.938710,1.000000,-7.347881e-16,7.071068e-01,-0.707107,15886.0,15921.0,15340.0,14988.0,...,15118.0,14903.0,14618.0,13692.0,12280.0,11681.0,11597.0,11692.0,11935.0,15386.0
2004-10-21 22:00:00,0.344707,-0.938710,1.000000,-7.347881e-16,8.660254e-01,-0.500000,15386.0,15886.0,15921.0,15340.0,...,15242.0,15118.0,14903.0,14618.0,13692.0,12280.0,11681.0,11597.0,11692.0,14437.0
2004-10-21 23:00:00,0.344707,-0.938710,1.000000,-7.347881e-16,9.659258e-01,-0.258819,14437.0,15386.0,15886.0,15921.0,...,15375.0,15242.0,15118.0,14903.0,14618.0,13692.0,12280.0,11681.0,11597.0,13446.0
2004-10-22 00:00:00,0.360771,-0.932655,0.623490,7.818315e-01,1.000000e+00,0.000000,13446.0,14437.0,15386.0,15886.0,...,15404.0,15375.0,15242.0,15118.0,14903.0,14618.0,13692.0,12280.0,11681.0,12627.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-02 18:00:00,-0.861702,-0.507415,-0.222521,9.749279e-01,-1.836970e-16,-1.000000,19585.0,19749.0,19921.0,19797.0,...,14913.0,14223.0,13771.0,13724.0,14072.0,14408.0,15101.0,16488.0,17815.0,19151.0
2017-08-02 19:00:00,-0.861702,-0.507415,-0.222521,9.749279e-01,2.588190e-01,-0.965926,19151.0,19585.0,19749.0,19921.0,...,15777.0,14913.0,14223.0,13771.0,13724.0,14072.0,14408.0,15101.0,16488.0,18495.0
2017-08-02 20:00:00,-0.861702,-0.507415,-0.222521,9.749279e-01,5.000000e-01,-0.866025,18495.0,19151.0,19585.0,19749.0,...,16471.0,15777.0,14913.0,14223.0,13771.0,13724.0,14072.0,14408.0,15101.0,17771.0


In [60]:
X_train=train_df.drop('target',axis=1)
y_train=train_df[['target']]

In [64]:
from sklearn.preprocessing import PowerTransformer
power_Tx=PowerTransformer().fit(X_train)
power_Ty=PowerTransformer().fit(y_train)
X_trainp=power_Tx.transform(X_train)
y_trainp=power_Ty.transform(y_train)

In [68]:
from xgboost import XGBRegressor 
from sklearn.model_selection import train_test_split
X_fit,X_val,y_fit,y_val=train_test_split(X_trainp,y_trainp,test_size=0.3)
# Bench model 
model=XGBRegressor(objective='reg:absoluteerror',tree_method='hist',n_estimators=10000,learning_rate=0.1,max_depth=5,eval_metric='mae').fit(X_val,y_val,eval_set=[(X_val,y_val)])

[0]	validation_0-mae:0.34963
[1]	validation_0-mae:0.20343
[2]	validation_0-mae:0.16379
[3]	validation_0-mae:0.14181
[4]	validation_0-mae:0.12844
[5]	validation_0-mae:0.12232
[6]	validation_0-mae:0.11859
[7]	validation_0-mae:0.11548
[8]	validation_0-mae:0.11342
[9]	validation_0-mae:0.10269
[10]	validation_0-mae:0.09824
[11]	validation_0-mae:0.09538
[12]	validation_0-mae:0.09310
[13]	validation_0-mae:0.09162
[14]	validation_0-mae:0.09036
[15]	validation_0-mae:0.08952
[16]	validation_0-mae:0.08879
[17]	validation_0-mae:0.08756
[18]	validation_0-mae:0.08704
[19]	validation_0-mae:0.08569
[20]	validation_0-mae:0.08508
[21]	validation_0-mae:0.08455
[22]	validation_0-mae:0.08384
[23]	validation_0-mae:0.08322
[24]	validation_0-mae:0.08219
[25]	validation_0-mae:0.08183
[26]	validation_0-mae:0.08071
[27]	validation_0-mae:0.08015
[28]	validation_0-mae:0.07965
[29]	validation_0-mae:0.07850
[30]	validation_0-mae:0.07792
[31]	validation_0-mae:0.07760
[32]	validation_0-mae:0.07725
[33]	validation_0-ma

In [75]:
pd.to_datetime(pd.Series(df_prpocessed.index))+timedelta(hours=1)

0        2004-10-29 01:00:00
1        2004-10-29 02:00:00
2        2004-10-29 03:00:00
3        2004-10-29 04:00:00
4        2004-10-29 05:00:00
                 ...        
111691   2017-07-27 19:00:00
111692   2017-07-27 20:00:00
111693   2017-07-27 21:00:00
111694   2017-07-27 22:00:00
111695   2017-07-27 23:00:00
Name: date_time, Length: 111696, dtype: datetime64[ns]

In [77]:
pd.to_datetime(pd.Series(df_prpocessed.index))

0        2004-10-29 00:00:00
1        2004-10-29 01:00:00
2        2004-10-29 02:00:00
3        2004-10-29 03:00:00
4        2004-10-29 04:00:00
                 ...        
111691   2017-07-27 18:00:00
111692   2017-07-27 19:00:00
111693   2017-07-27 20:00:00
111694   2017-07-27 21:00:00
111695   2017-07-27 22:00:00
Name: date_time, Length: 111696, dtype: datetime64[ns]